In [1]:
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer)
import chromadb
from chromadb.utils import embedding_functions
import sys
import time
import csv
import torch
import os
import pandas
import ast

In [2]:
access_token="your-access-token"
model_name='AbhirajSinghRajpurohit/Llama-3.2-1B-karma-finetuned'

In [3]:
# data=pandas.read_csv('Content_Storage_df.csv')
# embedding_df=pandas.DataFrame(data)
# embedding_df.head(1)
# chroma_client = chromadb.PersistentClient(path="./chroma_db")
# collection = chroma_client.get_or_create_collection("karma_embeddings")

# BATCH_SIZE = 166  # ChromaDB's max batch size

# def insert_data(df):
#     # Ensure embeddings are in the correct format (list of floats)
#     df["embedding_vector"] = df["embedding_vector"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    
#     # Convert verse_id to float format for uniqueness
#     df["verse_id"] = df.apply(lambda row: float(f"{row['chapter_id']}.{row['verse_id']}"), axis=1)

#     # Insert data in batches
#     for i in range(0, len(df), BATCH_SIZE):
#         batch = df.iloc[i : i + BATCH_SIZE]

#         collection.add(
#             ids=batch["verse_id"].astype(str).tolist(),      # Convert verse_id to string
#             documents=batch["verse_text"].tolist(),          # List of verses
#             embeddings=batch["embedding_vector"].tolist()    # List of embeddings
#         )
#         print(f"Inserted {len(batch)} records...")

#     print("✅ All data inserted successfully!")

# # Call function to insert your data
# insert_data(embedding_df)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name,token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_name,token=access_token)
tokenizer.pad_token = tokenizer.eos_token

chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_collection("karma_embeddings")

user_db = 'user_data.csv'

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

C:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ACER\.cache\huggingface\hub\models--AbhirajSinghRajpurohit--Llama-3.2-1B-karma-finetuned. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [17]:
def retrieve_context(question, top_k=1):
    embedding_fn = embedding_functions.DefaultEmbeddingFunction()
    question_embedding = embedding_fn([question])[0]

    # Retrieve top-k matching documents
    results = collection.query(
        query_embeddings=[question_embedding],
        n_results=top_k
    )
    if results["documents"]:
        # print(results["documents"])
        flat_documents = [doc for sublist in results["documents"] for doc in sublist]
        return " ".join(flat_documents) if flat_documents else "No relevant context found."
    
    return "No relevant context found."

def generate_answer(question, context):
    input_text = f"Question: {question} Context: {context} Answer:"
    input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"].to(model.device)

    output = model.generate(
        input_ids,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,  # Stops when EOS is generated
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    # answer = generated_text.replace(input_text, "").strip()
    answer = generated_text.split('Answer:')[1]
    print()
    print('**RAW-OUTPUT-GENERATED-BY-MODEL** : '+ generated_text)
    print()
    return answer



In [ ]:
while True:
    escapers= ['exit']
    question = input('**USER** : ')
    context = retrieve_context(question)
    answer= generate_answer(question,context)
    final_answer ='**EXTRACTED ANSWER FROM MODEL OUTPUT** : '+ answer
    if question.lower() in escapers:
        break
    for char in final_answer:
        sys.stdout.write(char)  # Write character without newline
        sys.stdout.flush()      # Force immediate output
        time.sleep(0.1)         # Adjust speed
    print()

USER :   What should a wise man not do?



RAW-OUTPUT-GENERATED-BY-MODEL: Question:  What should a wise man not do? Context: th brute force, and seek knowledge with out any effort and prosperity, by working the ruin of others, cannot be called wise it is foolishness to cut dowry a tree for its fruits a tree or a project that yields good fruits, should never be uprooted, o vipra how can i believe that a rich man to be an anchorite, and a drunken woman chaste trust not the untrustworthy nor confide any secret in your friend, lest he might betray you in a fit of anger a vast, deep and childlike faith in all, a universal clemency, and a close and watchful veiling of his own god like inherent virtues, are the traits which mark a noble soul the doer of an act does alone feel its consequence hence, all works should be coolly pondered over before execution the six things, such as the use of a new wine or trimulakam lit,horse radish in its three different forms, the partaking of curd in the night, or of dried meat, sleep during the day